# LAB-GPU EJERCICIO 1

### Librerias

In [1]:
import numpy as np
from multiprocessing import Pool
import time
from numba import njit, prange
from IPython import get_ipython

### Funciones

In [9]:
#Funcion usando el método secuencial

def reduc_operation(A):
    """Compute the sum of the elements of Array A in the range [0, value)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Funcion usando multiprocessing

def parallel_reduction(X, num_processes):
    """Perform parallel reduction using multiprocessing with num_processes."""
    n = len(X)
    chunk_size = n // num_processes
    sub_arrays = [X[i * chunk_size: (i + 1) * chunk_size] for i in range(num_processes - 1)]
    sub_arrays.append(X[(num_processes - 1) * chunk_size:])  # Add the last chunk
    
    with Pool(processes=num_processes) as pool:
        results = pool.map(reduc_operation, sub_arrays)
    
    return sum(results)

# Funcion usando @njit

@njit
def reduc_operation_njit(A):
    """Compute the sum of the elements of Array A using Numba (njit)."""
    s = 0
    for i in range(A.size):
        s += A[i]
    return s

# Funcion usando @njit(parallel=True)

@njit(parallel=True)
def reduc_operation_njit_parallel(A):
    """Compute the sum of the elements of Array A using Numba (parallel)."""
    s = 0
    for i in prange(A.size):
        s += A[i]
    return s


### Variables

In [10]:
# Definir la variable value

value = None

# Intentar obtener el valor de la variable VALUE pasada como un argumento
try:
    ipython = get_ipython()
    value = int(ipython.getoutput("echo $VALUE")[0])
except Exception as e:
    print(f"\nNo se pudo obtener el valor: {e}")
    value = 5*10**6  # Valor por defecto si no se define VALUE

print(f"\nRunning with {value} elements.")

X = np.random.rand(value)


No se pudo obtener el valor: invalid literal for int() with base 10: ''

Running with 5000000 elements.


### Código Inicial

In [11]:
# Utilizando en secuencial

tiempo = %timeit -r 2 -o -q reduc_operation(X)

print("\n Time taken by reduction operation using a function:", tiempo)

print(f"And the result of the sum of numbers in the range [0, value) is: {reduc_operation(X)}\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.sum()

tiempo = %timeit -r 2 -o -q np.sum(X)

print("Time taken by reduction operation using numpy.sum():", tiempo)

print("Now, the result using numpy.sum():", np.sum(X),"\n ")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")

# Utilizando numpy.ndarray.sum()

tiempo= %timeit -r 2 -o -q X.sum()

print("Time taken by reduction operation using numpy.ndarray.sum():", tiempo)

print("Now, the result using numpy.ndarray.sum():", X.sum(), "\n")

print("-----------------------------------------------------------------------------------------------------------------------------------------------\n")


 Time taken by reduction operation using a function: 4.74 s ± 146 ms per loop (mean ± std. dev. of 2 runs, 1 loop each)
And the result of the sum of numbers in the range [0, value) is: 2499665.155358141

-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.sum(): 6.37 ms ± 113 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.sum(): 2499665.1553585343 
 
-----------------------------------------------------------------------------------------------------------------------------------------------

Time taken by reduction operation using numpy.ndarray.sum(): 6.18 ms ± 7.73 µs per loop (mean ± std. dev. of 2 runs, 100 loops each)
Now, the result using numpy.ndarray.sum(): 2499665.1553585343 

-------------------------------------------------------------------------------------------------------------------------------

### Código Nuevo 

#### Utilizando procesos en paralelo

In [12]:
# Realizando 1 proceso

print("\n Utilizando 1 proceso:")

start_time = time.time()
result_1_process = parallel_reduction(X, 1)
time_1 = time.time() - start_time

print(f"Resultado: {result_1_process}, Tiempo: {time_1:.6f} segundos\n")

print("---------------------------------------------------------------------")

# Realizando 2 procesos

print("\n Utilizando 2 procesos:")

start_time = time.time()
result_2_process = parallel_reduction(X, 2)
time_2 = time.time() - start_time

print(f"Resultado: {result_2_process}, Tiempo: {time_2:.6f} segundos\n")

print("---------------------------------------------------------------------")

# Realizando 4 procesos

print("\n Utilizando 4 procesos:")

start_time = time.time()
result_4_process = parallel_reduction(X, 4)
time_4 = time.time() - start_time

print(f"Resultado: {result_4_process}, Tiempo: {time_4:.6f} segundos\n")

print("---------------------------------------------------------------------\n")


 Utilizando 1 proceso:
Resultado: 2499665.155358141, Tiempo: 4.839544 segundos

---------------------------------------------------------------------

 Utilizando 2 procesos:
Resultado: 2499665.1553585418, Tiempo: 2.669036 segundos

---------------------------------------------------------------------

 Utilizando 4 procesos:
Resultado: 2499665.1553585185, Tiempo: 1.451809 segundos

---------------------------------------------------------------------



#### Utilizando Numba

In [13]:
# Utilizando la función original

start_time = time.time()
result_original = reduc_operation(X)
time_original = time.time() - start_time

print("\n",f"Tiempo función original: {time_original:.6f} segundos, Resultado: {result_original}","\n")

print("-------------------------------------------------------------------------------------------------------------------\n")

# Utilizando la función con @njit

start_time = time.time()
result_njit = reduc_operation_njit(X)
time_njit = time.time() - start_time

print(f"Tiempo con @njit: {time_njit:.6f} segundos, Resultado: {result_njit}","\n")

print("-------------------------------------------------------------------------------------------------------------------\n")

# Utilizando la función @njit(parallel=True)

start_time = time.time()
result_njit_parallel = reduc_operation_njit_parallel(X)
time_njit_parallel = time.time() - start_time

print(f"Tiempo con @njit(parallel=True): {time_njit_parallel:.6f} segundos, Resultado: {result_njit_parallel}","\n")

print("-------------------------------------------------------------------------------------------------------------------\n")


 Tiempo función original: 4.813196 segundos, Resultado: 2499665.155358141 

-------------------------------------------------------------------------------------------------------------------

Tiempo con @njit: 0.527171 segundos, Resultado: 2499665.155358141 

-------------------------------------------------------------------------------------------------------------------

Tiempo con @njit(parallel=True): 1.907397 segundos, Resultado: 2499665.1553585413 

-------------------------------------------------------------------------------------------------------------------



### Comentario sobre los resultados obtenidos

#### **Resultados con Multiprocessing**
- **1 proceso:** Resultado: `2,499,665.155358141`, Tiempo: `4.839544 segundos`
- **2 procesos:** Resultado: `2,499,665.1553585418`, Tiempo: `2.669036 segundos`
- **4 procesos:** Resultado: `2,499,665.1553585185`, Tiempo: `1.451809 segundos`

**Observaciones:**  
- Al aumentar la cantidad de procesos, el tiempo de ejecución disminuye considerablemente, lo que confirma la efectividad de la paralelización.  
- Sin embargo, la reducción de tiempo tiene un límite debido a la sobrecarga asociada con la comunicación entre procesos.

#### **Resultados con Numba**
- **Función original (Python puro):** Resultado: `2,499,665.155358141`, Tiempo: `4.813196 segundos`
- **@njit:** Resultado: `2,499,665.155358141`, Tiempo: `0.527171 segundos`
- **@njit(parallel=True):** Resultado: `2,499,665.1553585413`, Tiempo: `1.907397 segundos`

**Observaciones:**  
- La optimización con `@njit` proporciona una mejora drástica en el tiempo de ejecución al convertir el código Python a código máquina eficiente.  
- Sorprendentemente, `@njit(parallel=True)` es menos eficiente que `@njit` en este caso, probablemente debido a la sobrecarga de paralelización para este tipo de problema.

#### **Conclusión General**
- `@njit` de Numba es el método más rápido y eficiente para esta tarea, superando ampliamente a Multiprocessing.  
- Multiprocessing es útil en casos donde Numba no es una opción, pero introduce mayor complejidad y sobrecarga.  
- La elección entre estos enfoques depende de la naturaleza del problema y los recursos disponibles. Para tareas como esta, Numba es la opción más recomendada.
